In [12]:
# ==============================================================================
# PROJET SCORING CRÉDIT - MODE "AUTONOME" (Liens H2O.ai stables)
# ==============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Bibliothèques Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc, classification_report

# Configuration visuelle
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
import warnings
warnings.filterwarnings('ignore')

print("🚀 DÉMARRAGE DU PROGRAMME...")

# --- ÉTAPE 1 : TÉLÉCHARGEMENT AUTOMATIQUE (SERVEURS STABLES) ---
# On utilise les liens de H2O.ai qui ne changent jamais
print("📥 Téléchargement des données depuis les serveurs H2O.ai...")

# Téléchargement Train
if not os.path.exists('cs-training.csv'):
    os.system('wget -q https://raw.githubusercontent.com/h2oai/h2o-2/master/smalldata/kaggle/GiveMeSomeCredit/cs-training.csv')

# Téléchargement Test
if not os.path.exists('cs-test.csv'):
    os.system('wget -q https://raw.githubusercontent.com/h2oai/h2o-2/master/smalldata/kaggle/GiveMeSomeCredit/cs-test.csv')

print("✅ Fichiers récupérés avec succès.")

# --- ÉTAPE 2 : CHARGEMENT ET NETTOYAGE ---
print("⚙️ Traitement des données...")

# Lecture
df_train = pd.read_csv("cs-training.csv")
df_test = pd.read_csv("cs-test.csv")

def clean_data(df, target_col='SeriousDlqin2yrs', is_train=True, imputer=None):
    # Suppression colonne index (souvent nommée Unnamed: 0)
    cols_to_drop = [c for c in df.columns if 'Unnamed' in c]
    df = df.drop(cols_to_drop, axis=1)

    if is_train:
        X = df.drop(target_col, axis=1)
        y = df[target_col]
        # Imputation par la médiane (robuste aux salaires extrêmes)
        imputer = SimpleImputer(strategy='median')
        X_imputed = imputer.fit_transform(X)
    else:
        # Pour le test, on enlève la cible si elle est présente
        if target_col in df.columns:
            X = df.drop(target_col, axis=1)
        else:
            X = df
        y = None
        # On utilise le même imputer
        X_imputed = imputer.transform(X)

    X_clean = pd.DataFrame(X_imputed, columns=X.columns)
    return X_clean, y, imputer

# Nettoyage
X, y, imputer_model = clean_data(df_train, is_train=True)
X_kaggle_test, _, _ = clean_data(df_test, is_train=False, imputer=imputer_model)

# Split (Train / Validation Interne)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# --- ÉTAPE 3 : IA (RANDOM FOREST) ---
print("🤖 Entraînement de l'IA (C'est la partie la plus longue)...")
# class_weight='balanced' est vital pour le crédit scoring
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced', n_jobs=-1)
model.fit(X_train, y_train)
print("✅ Modèle entraîné.")

# --- ÉTAPE 4 : VISUALISATION (Les 3 Graphes demandés) ---
print("\n📊 GÉNÉRATION DES GRAPHIQUES...")

# GRAPHE 1 : IMPORTANCE DES FEATURES
feature_importances = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
plt.figure(figsize=(10, 6))
sns.barplot(x=feature_importances, y=feature_importances.index, palette="viridis")
plt.title("🔍 GRAPHE 1 : Critères décisifs pour accorder un crédit", fontsize=14)
plt.xlabel("Importance (%)")
plt.tight_layout()
plt.show()

# GRAPHE 2 : MATRICE DE CONFUSION
y_pred = model.predict(X_val)
cm = confusion_matrix(y_val, y_pred)

plt.figure(figsize=(8, 6))
# On utilise des labels clairs pour le prof
group_names = ['Vrais Bons Payeurs', 'Faux Défauts (Erreur)', 'Faux Bons Payeurs (DANGER)', 'Vrais Défauts']
group_counts = ["{0:0.0f}".format(value) for value in cm.flatten()]
labels = [f"{v1}\n{v2}" for v1, v2 in zip(group_names, group_counts)]
labels = np.asarray(labels).reshape(2,2)

sns.heatmap(cm, annot=labels, fmt='', cmap='Blues', cbar=False, annot_kws={"size": 14})
plt.title('📉 GRAPHE 2 : Matrice de Confusion (Performance Réelle)', fontsize=16)
plt.ylabel('RÉALITÉ', fontsize=12)
plt.xlabel('PRÉDICTION DE L\'IA', fontsize=12)
plt.show()

# GRAPHE 3 : COURBE ROC
y_prob = model.predict_proba(X_val)[:, 1]
fpr, tpr, _ = roc_curve(y_val, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.title('📈 GRAPHE 3 : Courbe ROC', fontsize=14)
plt.xlabel('Taux de Faux Positifs')
plt.ylabel('Taux de Vrais Positifs')
plt.legend(loc="lower right")
plt.show()

# --- ÉTAPE 5 : FICHIER DE SOUMISSION ---
print("📝 Génération du fichier final...")
final_probs = model.predict_proba(X_kaggle_test)[:, 1]

submission = pd.DataFrame({
    "Id": range(1, len(final_probs) + 1),
    "Probability": final_probs
})

submission.to_csv("submission.csv", index=False)

print("\n" + "="*50)
print("✅ TERMINÉ ! Tout est vert.")
print("1. Les données ont été téléchargées automatiquement.")
print("2. Les graphes sont affichés ci-dessus.")
print("3. Le fichier 'submission.csv' est disponible dans le dossier à gauche.")
print("="*50)

🚀 DÉMARRAGE DU PROGRAMME...
📥 Téléchargement des données depuis les serveurs H2O.ai...
✅ Fichiers récupérés avec succès.
⚙️ Traitement des données...


ParserError: Error tokenizing data. C error: Expected 1 fields in line 42, saw 2
